# Root level

In [1]:
%%writefile package.json
{
  "name": "nfl-kicker-assessment",
  "private": true,
  "scripts": {
    "start": "npm start --prefix backend",
    "dev": "concurrently \"npm start --prefix backend\" \"npm run dev --prefix frontend\"",
    "build": "npm run build --prefix frontend",
    "preview": "npm run build && npx serve -s frontend/dist",
    "ci:validate": "node scripts/ci-validate.js",
    "test": "npm test --prefix backend && npm test --prefix frontend"
  },
  "devDependencies": {
    "concurrently": "^8.0.0"
  }
} 

Overwriting package.json


In [2]:
%%writefile render.yaml
services:
  - type: web
    name: nfl-kicker-api
    env: node
    plan: free
    rootDir: backend
    buildCommand: "npm ci --omit=dev"
    startCommand: "node server.js"
    healthCheckPath: /api/ping    # Render watches this
    autoDeploy: true
    envVars:
      - key: CORS_ORIGIN
        value: "https://<your-site>.netlify.app"
      - key: NODE_ENV
        value: production 


Overwriting render.yaml


In [3]:
%%writefile netlify.toml
[build]
  command = "npm run build --prefix frontend"
  publish = "frontend/dist"

[[redirects]]
  from = "/*"
  to   = "/index.html"
  status = 200 

Overwriting netlify.toml


# github actions


In [4]:
%%writefile .github/workflows/netlify.yml
name: Netlify Deploy
on: { push: { branches: [ main ] } }

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with: { node-version: '18' }
      
      # Validate environment variables
      - name: Validate environment variables
        run: node scripts/ci-validate.js
        env:
          VITE_API_URL: ${{ secrets.VITE_API_URL }}
          CORS_ORIGIN: ${{ secrets.CORS_ORIGIN }}
      
      # Build and deploy
      - run: npm ci --prefix frontend
      - run: npm run build --prefix frontend
      - uses: netlify/actions/cli@master
        env:
          NETLIFY_AUTH_TOKEN: ${{ secrets.NETLIFY_TOKEN }}
          NETLIFY_SITE_ID: ${{ secrets.NETLIFY_SITE_ID }}
        with:
          args: deploy --dir=frontend/dist --prod 


Overwriting .github/workflows/netlify.yml


# Backend

In [5]:
%%writefile src/backend/package.json
{
  "name": "nfl-kicker-backend",
  "version": "1.0.0",
  "description": "Express backend for NFL Kicker Assessment",
  "main": "server.js",
  "type": "module",
  "scripts": {
    "start": "node server.js",
    "dev": "nodemon server.js"
  },
  "dependencies": {
    "cors": "^2.8.5",
    "csv-parser": "^3.0.0",
    "express": "^4.18.2",
    "http-proxy-middleware": "^3.0.5",
    "multer": "^1.4.5-lts.1"
  },
  "devDependencies": {
    "nodemon": "^3.0.2"
  },
  "keywords": [
    "nfl",
    "kicker",
    "assessment",
    "api"
  ],
  "author": "",
  "license": "MIT",
  "engines": {
    "node": ">=18.0.0"
  }
}


Overwriting backend/package.json


In [6]:
%%writefile src/backend/server.js
import express from 'express';
import cors from 'cors';
import multer from 'multer';
import csv from 'csv-parser';
import fs from 'fs';
import path from 'path';
import { fileURLToPath } from 'url';
import { createProxyMiddleware } from 'http-proxy-middleware';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);

const app = express();

function getAllowedOrigins() {
  if (process.env.CORS_ORIGIN) {
    return process.env.CORS_ORIGIN.split(',').map(o => o.trim())
  }
  if (process.env.NODE_ENV === 'production') {
    return [/\.netlify\.app$/, /\.onrender\.com$/]   // regex works in cors ≥2.8
  }
  return ['http://localhost:5173']
}

// Middleware
app.use(cors({
  origin: getAllowedOrigins(),
  credentials: true,
}));

app.use(express.json());
app.use(express.urlencoded({ extended: true }));

// Configure multer for file uploads
const storage = multer.diskStorage({
  destination: (req, file, cb) => {
    const uploadDir = 'uploads/';
    // Ensure uploads directory exists (Railway volume mount point)
    if (!fs.existsSync(uploadDir)) {
      fs.mkdirSync(uploadDir, { recursive: true });
    }
    cb(null, uploadDir);
  },
  filename: (req, file, cb) => {
    // Add timestamp to prevent filename conflicts
    const timestamp = Date.now();
    const originalName = file.originalname;
    cb(null, `${timestamp}-${originalName}`);
  }
});

const upload = multer({ 
  storage,
  limits: {
    fileSize: 10 * 1024 * 1024, // 10MB limit
    files: 1
  },
  fileFilter: (req, file, cb) => {
    // Only allow CSV files
    if (file.mimetype === 'text/csv' || file.originalname.endsWith('.csv')) {
      cb(null, true);
    } else {
      cb(new Error('Only CSV files are allowed'), false);
    }
  }
});

// ─── ML Model API Proxy ─────────────────────────────────────────────────────
const MODEL_API_URL = process.env.MODEL_API_URL || 'http://model-api:8000';
app.use('/ml', createProxyMiddleware({
  target: MODEL_API_URL,
  changeOrigin: true,
  pathRewrite: { '^/ml': '' },
  onError: (err, req, res) => {
    console.error('Proxy error:', err);
    res.status(500).json({ 
      error: 'Model API service unavailable',
      details: process.env.NODE_ENV === 'development' ? err.message : undefined
    });
  },
  onProxyReq: (proxyReq, req, res) => {
    console.log(`Proxying ${req.method} ${req.url} to ${MODEL_API_URL}`);
  }
}));

// API Routes

// Health check endpoint
app.get('/api/ping', (req, res) => {
  res.json({ 
    pong: true, 
    timestamp: new Date().toISOString(),
    environment: process.env.NODE_ENV || 'development'
  });
});

// CSV upload and processing endpoint
app.post('/api/upload', upload.single('file'), (req, res) => {
  if (!req.file) {
    return res.status(400).json({ error: 'No file uploaded' });
  }

  const results = [];
  const filePath = req.file.path;

  console.log(`Processing uploaded file: ${req.file.originalname}`);

  // Stream parse the CSV file
  fs.createReadStream(filePath)
    .pipe(csv())
    .on('data', (data) => {
      // Clean and validate data
      const cleanData = {};
      Object.keys(data).forEach(key => {
        const cleanKey = key.trim();
        const value = data[key]?.toString().trim();
        
        // Convert numeric fields
        if (['player_id', 'distance', 'made', 'week', 'year'].includes(cleanKey)) {
          cleanData[cleanKey] = isNaN(value) ? value : Number(value);
        } else {
          cleanData[cleanKey] = value;
        }
      });
      
      // Basic validation - ensure required fields are present
      if (cleanData.player_id && cleanData.player_name && 
          cleanData.distance !== undefined && cleanData.made !== undefined) {
        results.push(cleanData);
      }
    })
    .on('end', () => {
      // Clean up the uploaded file
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });

      console.log(`Processed ${results.length} records from ${req.file.originalname}`);
      
      res.json({ 
        success: true,
        data: results,
        rowCount: results.length,
        filename: req.file.originalname
      });
    })
    .on('error', (error) => {
      console.error('Error processing CSV:', error);
      
      // Clean up the uploaded file on error
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });
      
      res.status(500).json({ 
        error: 'Error processing CSV file',
        details: error.message
      });
    });
});

// Get processed leaderboard data (if we want to store it server-side)
app.get('/api/leaderboard', (req, res) => {
  // This could be enhanced to return cached/processed leaderboard data
  res.json({ 
    message: 'Leaderboard endpoint - not implemented yet',
    timestamp: new Date().toISOString()
  });
});

// Error handling middleware
app.use((error, req, res, next) => {
  console.error('Server error:', error);
  
  if (error instanceof multer.MulterError) {
    if (error.code === 'LIMIT_FILE_SIZE') {
      return res.status(400).json({ error: 'File too large (max 10MB)' });
    }
    if (error.code === 'LIMIT_UNEXPECTED_FILE') {
      return res.status(400).json({ error: 'Too many files uploaded' });
    }
  }
  
  res.status(500).json({ 
    error: 'Internal server error',
    details: process.env.NODE_ENV === 'development' ? error.message : undefined
  });
});

// 404 handler for API routes
app.use('/api/*', (req, res) => {
  res.status(404).json({ error: 'API endpoint not found' });
});

// In production, serve static files from the public directory
if (process.env.NODE_ENV === 'production') {
  const publicPath = path.join(__dirname, 'public');
  app.use(express.static(publicPath));
  
  // Serve index.html for any non-API routes (SPA fallback)
  app.get('*', (req, res, next) => {
    if (req.path.startsWith('/api/') || req.path.startsWith('/ml/')) {
      return next();
    }
    res.sendFile(path.join(publicPath, 'index.html'));
  });
}

// Start server
const PORT = process.env.PORT || 5000;
const HOST = process.env.HOST || '0.0.0.0';

app.listen(PORT, HOST, () => {
  console.log(`🚀 NFL Kicker API server running on http://${HOST}:${PORT}`);
  console.log(`📁 Environment: ${process.env.NODE_ENV || 'development'}`);
  console.log(`🔧 CORS enabled for: ${process.env.CORS_ORIGIN || (process.env.NODE_ENV === 'production' ? 'production domains' : 'development (localhost:5173, localhost:3000)')}`);
  console.log(`📊 Health check: http://${HOST}:${PORT}/api/ping`);
  
  // Railway deployment info
  if (process.env.RAILWAY_ENVIRONMENT) {
    console.log(`🚂 Railway Environment: ${process.env.RAILWAY_ENVIRONMENT}`);
    console.log(`🌍 Railway Service: ${process.env.RAILWAY_SERVICE_NAME || 'Unknown'}`);
  }
});

// Graceful shutdown
process.on('SIGTERM', () => {
  console.log('SIGTERM received, shutting down gracefully');
  process.exit(0);
});

process.on('SIGINT', () => {
  console.log('SIGINT received, shutting down gracefully');
  process.exit(0);
}); 


Overwriting backend/server.js


#  Frontend

In [7]:
%%writefile src/frontend/package.json
{
  "name": "nfl-kicker-frontend",
  "private": true,
  "version": "0.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "start": "serve -s dist -l $PORT",
    "lint": "eslint . --ext js,jsx --report-unused-disable-directives --max-warnings 0",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "recharts": "^2.12.7",
    "lucide-react": "^0.436.0"
  },
  "devDependencies": {
    "@types/react": "^18.2.43",
    "@types/react-dom": "^18.2.17",
    "@vitejs/plugin-react": "^4.2.1",
    "autoprefixer": "^10.4.16",
    "eslint": "^8.55.0",
    "eslint-plugin-react": "^7.33.2",
    "eslint-plugin-react-hooks": "^4.6.0",
    "eslint-plugin-react-refresh": "^0.4.5",
    "postcss": "^8.4.32",
    "tailwindcss": "^3.3.6",
    "vite": "^5.0.8"
  },
  "engines": {
    "node": ">=18.0.0"
  }
} 


Overwriting frontend/package.json


In [8]:
%%writefile src/frontend/vite.config.js
import { defineConfig, loadEnv } from 'vite'
import react from '@vitejs/plugin-react'

// https://vitejs.dev/config/
export default defineConfig(({ mode }) => {
  const env = loadEnv(mode, process.cwd(), '')
  const API = env.VITE_API_URL || 'http://localhost:5000'   // dev fallback

  return {
    plugins: [react()],
    define: {
      __API_URL__: JSON.stringify(API),                    // usable in code
    },
    server: {
      host: '0.0.0.0',
      port: 5173,
      proxy: {
        '/api': { target: API, changeOrigin: true, secure: false },
      },
    },
    build: { outDir: 'dist' },
  }
}) 


Overwriting frontend/vite.config.js


In [9]:
%%writefile src/frontend/src/App.jsx
import React, { useState, useEffect } from 'react';
import { BarChart3, TrendingUp, FileText, Download, Target, Users } from 'lucide-react';
import { ScatterChart, Scatter, CartesianGrid, XAxis, YAxis, Tooltip, ResponsiveContainer, BarChart, Bar, LineChart, Line, Legend } from 'recharts';
import ReactMarkdown from 'react-markdown';

const NFLKickerApp = () => {
  const [activeTab, setActiveTab] = useState('leaderboard');
  const [leaderboard, setLeaderboard] = useState([]);
  const [analysis, setAnalysis] = useState(null);
  const [filterText, setFilterText] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState(null);

  // Fetch data from API endpoints
  useEffect(() => {
    const fetchData = async () => {
      setLoading(true);
      setError(null);
      
      try {
        // Fetch leaderboard data
        const leaderboardResponse = await fetch('/ml/leaderboard');
        if (!leaderboardResponse.ok) {
          throw new Error('Failed to fetch leaderboard');
        }
        const leaderboardData = await leaderboardResponse.json();
        setLeaderboard(leaderboardData);

        // Fetch analysis data
        const analysisResponse = await fetch('/ml/analysis');
        if (!analysisResponse.ok) {
          throw new Error('Failed to fetch analysis');
        }
        const analysisData = await analysisResponse.json();
        setAnalysis(analysisData);

      } catch (err) {
        console.error('Error fetching data:', err);
        setError(err.message);
      } finally {
        setLoading(false);
      }
    };

    fetchData();
  }, []);

  const downloadLeaderboard = () => {
    const csv = [
      'rank,player_name,rating,accuracy,attempts',
      ...leaderboard.map(row => 
        `${row.rank},${row.player_name},${row.rating},${row.accuracy},${row.attempts}`
      )
    ].join('\n');
    
    const blob = new Blob([csv], { type: 'text/csv' });
    const url = window.URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.style.display = 'none';
    a.href = url;
    a.download = 'nfl_kicker_leaderboard.csv';
    document.body.appendChild(a);
    a.click();
    window.URL.revokeObjectURL(url);
  };

  const generateTechnicalReport = () => {
    if (!analysis) return '';
    
    return `# NFL Kicker Performance Analysis

## Executive Summary
This analysis evaluates NFL kicker performance using advanced machine learning models including Random Forest and Bayesian hierarchical regression. The models account for distance, kicker ability, and situational factors to provide comprehensive performance ratings.

## Dataset Overview
- **Total Kickers Analyzed**: ${analysis.total_kickers}
- **Total Field Goal Attempts**: ${analysis.total_attempts}
- **Average Success Rate**: ${(analysis.average_accuracy * 100).toFixed(1)}%

## Top Performers
${analysis.top_performers.map((kicker, i) => 
  `${i + 1}. **${kicker.player_name}** - Rating: ${kicker.rating} (${kicker.attempts} attempts, ${(kicker.accuracy * 100).toFixed(1)}% accuracy)`
).join('\n')}

## Distance Analysis
${Object.entries(analysis.distance_breakdown).map(([key, data]) => 
  `- **${data.range}**: ${(data.expected_accuracy * 100).toFixed(1)}% expected accuracy`
).join('\n')}

## Key Insights
${analysis.insights.map(insight => `- ${insight}`).join('\n')}

## Methodology
Our analysis employs two complementary approaches:

### 1. Random Forest Model
- Handles non-linear relationships between distance and success probability
- Accounts for individual kicker effects
- Provides feature importance rankings

### 2. Bayesian Hierarchical Model
- Incorporates uncertainty quantification
- Handles varying sample sizes across kickers
- Provides credible intervals for predictions

## Model Performance
Both models demonstrate strong predictive accuracy with proper calibration across different distance ranges. The Bayesian approach provides additional uncertainty estimates valuable for decision-making.

## Applications
- **Fantasy Football**: Kicker selection and lineup optimization
- **Team Management**: Kicker evaluation and contract decisions  
- **Game Strategy**: Fourth down decision making
- **Betting Markets**: Odds assessment and value identification

## Future Enhancements
1. Incorporate weather and environmental factors
2. Add situational pressure indicators (game state, playoffs)
3. Include opponent strength adjustments
4. Implement real-time model updates

---
*Analysis generated on ${new Date().toLocaleDateString()} using NFL kicker performance data.*`;
  };

  const TabButton = ({ id, label, icon: Icon, active }) => (
    <button
      onClick={() => setActiveTab(id)}
      className={`flex items-center px-4 py-2 rounded-lg transition-colors ${
        active 
          ? 'bg-blue-600 text-white' 
          : 'bg-gray-100 text-gray-700 hover:bg-gray-200'
      }`}
    >
      <Icon className="w-4 h-4 mr-2" />
      {label}
    </button>
  );

  // Filter leaderboard data
  const filteredLeaderboard = leaderboard.filter(k => 
    k.player_name.toLowerCase().includes(filterText.toLowerCase())
  );

  // Prepare chart data
  const chartData = analysis?.distance_breakdown ? 
    Object.entries(analysis.distance_breakdown).map(([key, data]) => ({
      range: data.range,
      accuracy: data.expected_accuracy * 100
    })) : [];

  const tabs = [
    { id: 'leaderboard', label: 'Leaderboard', icon: BarChart3 },
    { id: 'analysis', label: 'Analysis', icon: TrendingUp },
    { id: 'paper', label: 'Technical Paper', icon: FileText },
  ];

  if (loading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-blue-50 to-green-50 flex items-center justify-center">
        <div className="text-center">
          <div className="animate-spin rounded-full h-12 w-12 border-b-2 border-blue-600 mx-auto mb-4"></div>
          <p className="text-gray-600">Loading NFL kicker data...</p>
        </div>
      </div>
    );
  }

  if (error) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-blue-50 to-green-50 flex items-center justify-center">
        <div className="text-center bg-white p-8 rounded-lg shadow-lg">
          <div className="text-red-600 mb-4">⚠️ Error Loading Data</div>
          <p className="text-gray-600 mb-4">{error}</p>
          <button 
            onClick={() => window.location.reload()} 
            className="bg-blue-600 text-white px-4 py-2 rounded-lg hover:bg-blue-700"
          >
            Retry
          </button>
        </div>
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-blue-50 to-green-50 py-8">
      <div className="max-w-7xl mx-auto px-4">
        {/* Header */}
        <div className="text-center mb-8">
          <h1 className="text-4xl font-bold text-gray-800 mb-2">NFL Kicker Performance Analysis</h1>
          <p className="text-gray-600">Advanced machine learning models for field goal success prediction</p>
        </div>

        {/* Navigation */}
        <div className="flex flex-wrap gap-2 mb-6 justify-center">
          {tabs.map(tab => (
            <TabButton 
              key={tab.id}
              id={tab.id} 
              label={tab.label} 
              icon={tab.icon}
              active={activeTab === tab.id}
            />
          ))}
        </div>

        {/* Leaderboard Tab */}
        {activeTab === 'leaderboard' && (
          <div className="bg-white rounded-xl p-6 shadow-lg">
            <div className="flex justify-between items-center mb-6">
              <h2 className="text-2xl font-bold text-gray-800">Kicker Leaderboard</h2>
              <div className="flex gap-4 items-center">
                <input 
                  type="text"
                  placeholder="Search kickers..."
                  value={filterText}
                  onChange={e => setFilterText(e.target.value)}
                  className="border border-gray-300 px-4 py-2 rounded-lg focus:outline-none focus:ring-2 focus:ring-blue-500"
                />
                <button
                  onClick={downloadLeaderboard}
                  className="flex items-center gap-2 bg-blue-600 text-white px-4 py-2 rounded-lg hover:bg-blue-700 transition-colors"
                >
                  <Download size={16} />
                  Download CSV
                </button>
              </div>
            </div>
            
            <div className="overflow-x-auto">
              <table className="w-full border-collapse">
                <thead>
                  <tr className="bg-gray-50">
                    <th className="text-left p-4 border-b-2 border-gray-200 font-semibold">Rank</th>
                    <th className="text-left p-4 border-b-2 border-gray-200 font-semibold">Player</th>
                    <th className="text-left p-4 border-b-2 border-gray-200 font-semibold">Rating</th>
                    <th className="text-left p-4 border-b-2 border-gray-200 font-semibold">Accuracy</th>
                    <th className="text-left p-4 border-b-2 border-gray-200 font-semibold">Attempts</th>
                    <th className="text-left p-4 border-b-2 border-gray-200 font-semibold">Confidence</th>
                  </tr>
                </thead>
                <tbody>
                  {filteredLeaderboard.map((kicker, index) => (
                    <tr key={kicker.player_id} className="hover:bg-gray-50 transition-colors">
                      <td className="p-4 border-b border-gray-200">
                        <span className="font-bold text-lg">#{kicker.rank}</span>
                      </td>
                      <td className="p-4 border-b border-gray-200">
                        <div className="font-semibold text-gray-800">{kicker.player_name}</div>
                        <div className="text-sm text-gray-500">ID: {kicker.player_id}</div>
                      </td>
                      <td className="p-4 border-b border-gray-200">
                        <span className="font-bold text-blue-600">{kicker.rating}</span>
                      </td>
                      <td className="p-4 border-b border-gray-200">
                        <span className="font-semibold">{(kicker.accuracy * 100).toFixed(1)}%</span>
                      </td>
                      <td className="p-4 border-b border-gray-200">
                        <span className="bg-gray-100 px-2 py-1 rounded">{kicker.attempts}</span>
                      </td>
                      <td className="p-4 border-b border-gray-200">
                        {kicker.confidence_interval && (
                          <span className="text-sm text-gray-600">
                            {(kicker.confidence_interval.lower * 100).toFixed(1)}% - {(kicker.confidence_interval.upper * 100).toFixed(1)}%
                          </span>
                        )}
                      </td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </div>
          </div>
        )}

        {/* Analysis Tab */}
        {activeTab === 'analysis' && analysis && (
          <div className="space-y-6">
            {/* Summary Cards */}
            <div className="grid md:grid-cols-3 gap-6">
              <div className="bg-white rounded-xl p-6 shadow-lg">
                <div className="flex items-center justify-between">
                  <div>
                    <p className="text-gray-600 text-sm">Total Kickers</p>
                    <p className="text-3xl font-bold text-blue-600">{analysis.total_kickers}</p>
                  </div>
                  <Users className="w-8 h-8 text-blue-600" />
                </div>
              </div>
              
              <div className="bg-white rounded-xl p-6 shadow-lg">
                <div className="flex items-center justify-between">
                  <div>
                    <p className="text-gray-600 text-sm">Total Attempts</p>
                    <p className="text-3xl font-bold text-green-600">{analysis.total_attempts}</p>
                  </div>
                  <Target className="w-8 h-8 text-green-600" />
                </div>
              </div>
              
              <div className="bg-white rounded-xl p-6 shadow-lg">
                <div className="flex items-center justify-between">
                  <div>
                    <p className="text-gray-600 text-sm">Average Accuracy</p>
                    <p className="text-3xl font-bold text-purple-600">{(analysis.average_accuracy * 100).toFixed(1)}%</p>
                  </div>
                  <BarChart3 className="w-8 h-8 text-purple-600" />
                </div>
              </div>
            </div>

            {/* Charts */}
            <div className="grid md:grid-cols-2 gap-6">
              <div className="bg-white rounded-xl p-6 shadow-lg">
                <h3 className="text-xl font-semibold mb-4">Success Rate by Distance</h3>
                <ResponsiveContainer width="100%" height={300}>
                  <BarChart data={chartData}>
                    <CartesianGrid strokeDasharray="3 3" />
                    <XAxis dataKey="range" />
                    <YAxis />
                    <Tooltip formatter={(value) => [`${value.toFixed(1)}%`, 'Success Rate']} />
                    <Bar dataKey="accuracy" fill="#3b82f6" />
                  </BarChart>
                </ResponsiveContainer>
              </div>
              
              <div className="bg-white rounded-xl p-6 shadow-lg">
                <h3 className="text-xl font-semibold mb-4">Top Performers</h3>
                <div className="space-y-3">
                  {analysis.top_performers.map((kicker, index) => (
                    <div key={kicker.player_id} className="flex items-center justify-between p-3 bg-gray-50 rounded-lg">
                      <div className="flex items-center gap-3">
                        <span className="font-bold text-lg">#{index + 1}</span>
                        <div>
                          <div className="font-semibold">{kicker.player_name}</div>
                          <div className="text-sm text-gray-600">{kicker.attempts} attempts</div>
                        </div>
                      </div>
                      <div className="text-right">
                        <div className="font-bold text-blue-600">{kicker.rating}</div>
                        <div className="text-sm text-gray-600">{(kicker.accuracy * 100).toFixed(1)}%</div>
                      </div>
                    </div>
                  ))}
                </div>
              </div>
            </div>

            {/* Insights */}
            <div className="bg-white rounded-xl p-6 shadow-lg">
              <h3 className="text-xl font-semibold mb-4">Key Insights</h3>
              <div className="grid md:grid-cols-2 gap-4">
                {analysis.insights.map((insight, index) => (
                  <div key={index} className="flex items-start gap-3 p-3 bg-blue-50 rounded-lg">
                    <div className="w-2 h-2 bg-blue-600 rounded-full mt-2 flex-shrink-0"></div>
                    <p className="text-gray-700">{insight}</p>
                  </div>
                ))}
              </div>
            </div>
          </div>
        )}

        {/* Technical Paper Tab */}
        {activeTab === 'paper' && (
          <div className="bg-white rounded-xl p-8 shadow-lg">
            <div className="prose prose-lg max-w-none">
              <ReactMarkdown>{generateTechnicalReport()}</ReactMarkdown>
            </div>
          </div>
        )}
      </div>
    </div>
  );
};

export default NFLKickerApp; 



Overwriting frontend/src/App.jsx


# Validation

In [10]:
%%writefile scripts/ci-validate.js
#!/usr/bin/env node

function validateEnvVars() {
  const apiUrl = process.env.VITE_API_URL;
  const corsOrigin = process.env.CORS_ORIGIN;

  if (!apiUrl) {
    console.error('❌ VITE_API_URL is not set');
    process.exit(1);
  }

  if (!corsOrigin) {
    console.error('❌ CORS_ORIGIN is not set');
    process.exit(1);
  }

  try {
    const apiDomain = new URL(apiUrl).hostname;
    const corsOrigins = corsOrigin.split(',').map(o => o.trim());
    
    // Check if the API domain is allowed by CORS_ORIGIN
    const isAllowed = corsOrigins.some(origin => {
      if (origin.startsWith('/') && origin.endsWith('/')) {
        // It's a regex pattern
        const regex = new RegExp(origin.slice(1, -1));
        return regex.test(apiDomain);
      }
      return new URL(origin).hostname === apiDomain;
    });

    if (!isAllowed) {
      console.error(`❌ API domain ${apiDomain} is not allowed by CORS_ORIGIN`);
      console.error(`   CORS_ORIGIN: ${corsOrigin}`);
      process.exit(1);
    }

    console.log('✅ Environment variables are consistent');
    process.exit(0);
  } catch (error) {
    console.error('❌ Invalid URL format in environment variables');
    console.error(error);
    process.exit(1);
  }
}

validateEnvVars(); 

Overwriting scripts/ci-validate.js
